In [1]:
# This notebook is for generating molecular electrostatic maps of each molecule and moving it to its class directory.

In [2]:
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem.Draw import SimilarityMaps
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.preprocessing import LabelEncoder
import os
import numpy as np
from PIL import Image
import shutil

In [4]:
# Change the path according to your directory.
df1 = pd.read_csv('final_total', sep=' ', na_values=['na'])
df1 = df1.dropna(ignore_index=True)
df1.loc[:, 'ido_ic50'] = (np.log10((df1.loc[:, 'ido_ic50'])*10**(-9)))*(-1)
df1.loc[:, 'tdo_ic50'] = (np.log10((df1.loc[:, 'tdo_ic50'])*10**(-9)))*(-1)
df1

,CHEMBLID,smiles,ido_ic50,tdo_ic50
0,CHEMBL1098875,O=C1c2cc(F)ccc2-n2c1nc1ccccc1c2=O,7.638272,7.221849
1,CHEMBL1209728,Cc1c(Br)oc2c1C(=O)C(=O)c1c-2ccc2c1CCCC2(C)C,5.000000,5.000000
2,CHEMBL1276265,O=C1c2ccc(Cl)cc2-n2c1nc1ccccc1c2=O,6.737549,3.649364
3,CHEMBL1346056,Oc1ccccc1-c1nc2c3ccccc3c3ccccc3c2[nH]1,4.928486,4.911864
4,CHEMBL139935,O=[N+]([O-])c1cc(F)c2cccnc2c1O,4.835350,4.519562
...,...,...,...,...
755,CHEMBL5221020,O[C@@H]1CCCC[C@H]1CNc1c(Br)ccc2[nH]ncc12,5.910095,4.949234
756,CHEMBL5221102,Brc1cc(NC[C@H]2CCCN2)c2cn[nH]c2c1,4.248336,5.127844
757,CHEMBL578036,CC1(C)CCCc2c1ccc1c2C(=O)C(=O)c2c(CO)coc2-1,5.000000,6.346787
758,CHEMBL584991,Nc1nonc1/C(=N/O)Nc1ccc(F)c(Cl)c1,7.301030,5.000000


In [5]:
df2 = df1.loc[:, ['ido_ic50', 'tdo_ic50']]
df2

,ido_ic50,tdo_ic50
0,7.638272,7.221849
1,5.000000,5.000000
2,6.737549,3.649364
3,4.928486,4.911864
4,4.835350,4.519562
...,...,...
755,5.910095,4.949234
756,4.248336,5.127844
757,5.000000,6.346787
758,7.301030,5.000000


In [6]:
n = 0
for i in range(len(df2)):
    if df2['ido_ic50'].values[i] >= 6.15:
        n += 1
print(n)

379


In [7]:
n = 0
for i in range(len(df2)):
    if df2['tdo_ic50'].values[i] >= 6:
        n += 1
print(n)

288


In [8]:
for i in range(len(df2)):
    if df2['ido_ic50'].values[i] >= 6.15:
        df2.loc[i, ['ido_ic50']] = 1
    else:
        df2.loc[i, ['ido_ic50']] = 0

In [9]:
for i in range(len(df2)):
    if df2['tdo_ic50'].values[i] >= 6.0:
        df2.loc[i, ['tdo_ic50']] = 1
    else:
        df2.loc[i, ['tdo_ic50']] = 0

In [10]:
df2

,ido_ic50,tdo_ic50
0,1.0,1.0
1,0.0,0.0
2,1.0,0.0
3,0.0,0.0
4,0.0,0.0
...,...,...
755,0.0,0.0
756,0.0,0.0
757,0.0,1.0
758,1.0,0.0


In [11]:
newcol = []
for i in range(len(df2)):
    if df2['ido_ic50'].values[i] == 1.0 and df2['tdo_ic50'].values[i] == 1.0:
        newcol.append('AA')
    elif df2['ido_ic50'].values[i] == 0.0 and df2['tdo_ic50'].values[i] == 0.0:
        newcol.append('II')
    elif df2['ido_ic50'].values[i] == 0.0 and df2['tdo_ic50'].values[i] == 1.0:
        newcol.append('IA')
    else:
        newcol.append('AI')

In [12]:
target = pd.DataFrame(data=newcol, columns=['ido_tdo'])
target

,ido_tdo
0,AA
1,II
2,AI
3,II
4,II
...,...
755,II
756,II
757,IA
758,AI


In [13]:
y = target.values.reshape(len(df2),)
y.shape

(760,)

In [14]:
encoder = LabelEncoder()

In [15]:
y_label = encoder.fit_transform(y)

In [16]:
y_label

array([0, 3, 1, 3, 3, 3, 2, 3, 3, 3, 2, 3, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1,
       3, 3, 3, 3, 3, 3, 3, 2, 2, 3, 0, 2, 3, 3, 1, 2, 1, 2, 3, 3, 2, 3,
       3, 2, 2, 2, 2, 0, 2, 0, 2, 2, 2, 2, 2, 2, 2, 3, 1, 3, 3, 3, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 0, 1, 3, 1, 1, 1, 1, 1, 1, 1, 1,
       3, 1, 1, 3, 1, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 2, 0, 1, 3, 0, 0,
       1, 3, 0, 2, 1, 3, 3, 1, 0, 1, 0, 0, 0, 0, 1, 3, 1, 0, 0, 0, 1, 0,
       3, 1, 0, 0, 1, 0, 2, 0, 0, 0, 1, 0, 0, 1, 1, 3, 0, 0, 3, 0, 0, 3,
       0, 1, 2, 1, 0, 0, 0, 0, 2, 2, 1, 0, 1, 0, 3, 0, 0, 0, 0, 0, 3, 0,
       0, 1, 1, 2, 0, 0, 0, 1, 0, 0, 3, 1, 1, 2, 0, 3, 3, 0, 0, 3, 0, 3,
       0, 3, 3, 3, 0, 0, 0, 1, 1, 1, 1, 0, 2, 0, 3, 0, 0, 0, 1, 1, 1, 0,
       1, 0, 0, 0, 3, 0, 3, 3, 3, 2, 2, 0, 1, 0, 2, 3, 0, 0, 1, 0, 0, 0,
       3, 3, 0, 0, 3, 3, 1, 2, 3, 0, 3, 0, 2, 0, 1, 3, 1, 0, 0, 3, 3, 0,
       0, 2, 1, 0, 3, 0, 3, 3, 0, 3, 3, 1, 1, 3, 3,

In [17]:
from rdkit.Chem import SaltRemover as sr
remover = sr.SaltRemover()

In [18]:
mols = []
for i in range(len(df1)):
    try:
        mol_i = Chem.MolFromSmiles(df1['smiles'][i])
        if mol_i is None:
            print(f"[WARNING] Invalid SMILES at index {i}, skipping.")
            mols.append(None)  # Maintain indexing for multiprocessing
            continue
        mol_i = remover.StripMol(mol_i, dontRemoveEverything=True)
        mols.append(mol_i)
    except Exception as e:
        print(e)
len(mols)

760

In [19]:
xborder_size = 100
yborder_size = 50
def crop_image(image):
    # Crop the image by removing 'border_size' pixels from all sides
    return image[xborder_size:-xborder_size, yborder_size:-yborder_size]

In [20]:
os.makedirs('./maps_git', exist_ok=True)
os.chdir('./maps_git')
os.makedirs('AA'), os.makedirs('AI'), os.makedirs('IA'), os.makedirs('II')

for k in range(len(df1)):
    try:
        AllChem.ComputeGasteigerCharges(mols[k])
        contribs = [mols[k].GetAtomWithIdx(i).GetDoubleProp('_GasteigerCharge') for i in range(mols[k].GetNumAtoms())]
        fig = SimilarityMaps.GetSimilarityMapFromWeights(mols[k], contribs, colorMap='jet', contourLines=10)
        fig.savefig(f"fig_{k}", bbox_inches='tight', pad_inches=0)
        img = Image.open(f"fig_{k}.png").convert('RGB')
        img = np.array(img)
        img = Image.fromarray(crop_image(img))
        img.save(f"fig_{k}.png")
        plt.close()
        if y_label[k] == 0:
            shutil.move(f"fig_{k}.png", './AA')
        elif y_label[k] == 1:
            shutil.move(f"fig_{k}.png", './AI')
        elif y_label[k] == 2:
            shutil.move(f"fig_{k}.png", './IA')
        elif y_label[k] == 3:
            shutil.move(f"fig_{k}.png", './II')
    except Exception as e:
        print(e)
        